In [43]:
import csv
import os
import tqdm

import biosppy.signals.ecg as ecg
import biosppy
import neurokit2 as nk

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp

from scipy.fftpack import fft
import scipy.fftpack as sf

import sys

sys.path.append("..")
from util import load_base_data


SAMPLING_RATE = 300.0
DATAPATH = "/Users/ericschreiber/dev/ETH/AML/Project_1/aml-2023/task2/data"

In [2]:
X_train, y_train, X_test = load_base_data()

In [101]:
freq_array = [
    0.46981001,
    0.94989355,
    1.34722766,
    1.92037653,
    2.45140581,
    3.58897684,
    5.61198292,
    7.85416366,
    11.77742192,
    19.56926069,
]


def sum_freq_bands_for_beat(heartbeat, n=10, freq_array=freq_array):
    fourier_specture = np.abs(fft(heartbeat))
    freqs = sf.fftfreq(len(fourier_specture), 1.0 / SAMPLING_RATE)
    fourier_specture = fourier_specture[freqs >= 0]
    freqs = freqs[freqs >= 0]

    # cut even more base on the freq_array
    fourier_specture = fourier_specture[freqs <= freq_array[-1]]
    freqs = freqs[freqs <= freq_array[-1]]

    # compute the sums of frequency bands
    sums = []
    sums.append(np.sum(fourier_specture[freqs <= freq_array[0]]))
    for i in range(len(freq_array) - 1):
        sum = np.sum(
            fourier_specture[
                np.logical_and(freqs > freq_array[i], freqs <= freq_array[i + 1])
            ]
        )
        sums.append(sum)

    return sums


def sum_freq_bands_for_signal(clean_signal):
    _, info = nk.ecg_peaks(ecg_cleaned=clean_signal, sampling_rate=SAMPLING_RATE)
    features = []
    n_peaks = 10
    rpeaks = info["ECG_R_Peaks"]
    beats = biosppy.signals.ecg.extract_heartbeats(
        signal=clean_signal, rpeaks=rpeaks, sampling_rate=SAMPLING_RATE
    )["templates"]
    n_beats = len(beats)
    for i in range(n_beats):
        features.append(np.array(sum_freq_bands_for_beat(beats[i], n_peaks)))
    features = list(np.array(features).T)
    return features


def create_features_from_signal(ecg_or_ftt):
    features = []
    for sig in ecg_or_ftt:
        if len(sig) > 0:
            mean = np.mean(sig)
            std = np.std(sig)
            median = np.median(sig)
            min = np.min(sig)
            max = np.max(sig)
            skew = sp.stats.skew(sig)
            kurtosis = sp.stats.kurtosis(sig)
            variation = sp.stats.variation(sig)
            iqr = sp.stats.iqr(sig)
            features += [mean, std, median, min, max, skew, kurtosis, variation, iqr]
        else:
            print("Problem with feature extraction: empty array")
            features += [0] * 90

    if len(features) != 90:
        print("Problem with feature extraction: wrong number of features")
        print(len(features))
        print(features)
        print(ecg_or_ftt)
        return [0] * 90
    return features

In [4]:
def signal_to_noise(ecg):
    try:
        mean = np.mean(ecg)
        std = np.std(ecg)
        db = 20 * np.log10(abs(mean / std))
    except:
        db = 0
    return [db]

In [5]:
def signal_over_70_percent(ecg, peaks):
    try:
        threshold = np.max(ecg) * 0.7
        over = np.sum(ecg > threshold)
        percent = over / len(ecg)
        percent_over_peaks = over / len(peaks)
    except:
        percent = 0
        percent_over_peaks = 0
    return [percent, percent_over_peaks]

In [51]:
def make_features(ecg):
    cleaned = nk.ecg_clean(ecg, sampling_rate=SAMPLING_RATE)
    _, info = nk.ecg_peaks(ecg_cleaned=cleaned, sampling_rate=SAMPLING_RATE)
    rpeaks = info["ECG_R_Peaks"]
    features = []
    features += signal_to_noise(cleaned)
    features += signal_over_70_percent(cleaned, rpeaks)
    features += create_features_from_signal(sum_freq_bands_for_signal(cleaned))
    features = np.array(features).flatten()
    return features


def make_features_from_df(df):
    features = []
    for i in tqdm.tqdm(range(len(df))):
        ecg = df.iloc[i].values
        features.append(make_features(ecg))
    numpy = np.array(features)
    df = pd.DataFrame(numpy).reset_index(drop=False)
    df.drop(columns=["index"], inplace=True)
    df.index.name = "id"

    return df

In [106]:
features_X_train = make_features_from_df(X_train)

  0%|          | 0/5117 [00:00<?, ?it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 1485 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9254 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9582 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9090 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/erics

Problem with feature extraction: wrong number of features
0
[]
[]


/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9114 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 10910 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 3609 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9189 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
 37%|███▋      | 1890/5117 [00:26<00:43, 73.72it/s]

Problem with feature extraction: wrong number of features
0
[]
[]


/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9267 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
 62%|██████▏   | 3162/5117 [00:43<00:25, 75.35it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9512 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9620 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9229 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/

In [107]:
features_X_train
# problematic index: 3145

,0,1,2,3,4,5,6,7,8,9,...,83,84,85,86,87,88,89,90,91,92
id,,,,,,,,,,,,,,,,,,,,,
0,-47.970890,0.003033,0.805970,3475.590125,2941.998865,2804.172958,244.126754,15593.248446,2.262259,5.957297,...,1556.578220,6951.862322,719.042546,6942.696832,5532.129757,8663.814998,0.226704,-0.492148,0.103432,923.393671
1,-29.847607,0.001516,0.750000,6835.840038,5421.949691,6138.581016,240.764174,22315.458578,1.011273,0.476087,...,1384.678636,5586.242881,784.276735,5623.145320,3967.134638,7248.781561,0.113381,-0.629048,0.140394,1117.135927
2,-74.317064,0.001348,0.800000,6903.411366,13892.612081,3624.357028,77.394504,79377.447724,4.716661,21.707970,...,1049.967430,15533.854725,706.717484,15536.030951,14336.382396,17815.518987,0.905013,1.696980,0.045495,731.022074
3,-51.919583,0.004998,1.271429,5170.687276,3688.461808,4397.644418,199.842901,14887.747898,1.076558,0.577835,...,3543.621824,17983.650181,2561.188968,17994.035406,10934.371074,23249.265311,-0.164513,-0.409203,0.142418,4044.295826
4,-43.771308,0.001741,0.673913,4879.268812,4128.254984,4352.920585,55.437931,18302.006479,1.439521,2.101722,...,1612.763687,9775.197489,1122.380427,9917.439361,6739.341157,12892.557473,-0.143870,0.920279,0.114819,1450.506437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,-44.085350,0.002471,2.095238,10242.425075,10801.358351,6544.482441,36.268460,40077.306616,1.636421,1.641021,...,3671.188792,15287.999391,4033.693636,15878.952854,3689.778768,22405.043269,-0.669186,1.373376,0.263847,3379.090130
5113,-80.590864,0.008985,5.000000,1836.742309,1220.902346,1955.233789,39.553873,4999.498932,0.591666,-0.093997,...,675.644684,6393.960646,528.526972,6450.580449,5369.007892,7530.572404,-0.113361,-0.626529,0.082660,755.805347
5114,-39.676188,0.004942,2.444444,6991.835149,4415.982441,6832.217985,630.995475,20639.813634,0.722650,0.540665,...,849.378416,14462.903728,836.089414,14304.655054,12689.503563,16796.806459,0.554145,0.554102,0.057809,992.506426


In [110]:
# How many infs
print(f"Number of infs: {np.sum(np.isinf(features_X_train), axis=0).sum()}")
# Which columns have infs
cols_with_infs = np.where(np.isinf(features_X_train))[1]
cols_with_infs_unique = np.unique(cols_with_infs)
print(f"Columns with infs: {len(cols_with_infs_unique)}")
print(
    f"biggest pos value except inf: {np.max(features_X_train[features_X_train != np.inf].max())}"
)
print(f"biggest neg value: {np.min(features_X_train).min()}")
biggest_pos = np.max(features_X_train[features_X_train != np.inf].max())
biggest_neg = np.min(features_X_train).min()
# Replace infs with biggest pos value
features_X_train[features_X_train == np.inf] = biggest_pos
# Replace -infs with biggest neg value
features_X_train[features_X_train == -np.inf] = biggest_neg

Number of infs: 2
Columns with infs: 1
biggest pos value except inf: 430224.5726202455
biggest neg value: -115.26008220042007


In [112]:
X_train_save_path = os.path.join(
    DATAPATH, "feature_extraction/spectral_analysis_X_train_features.csv"
)
features_X_train.to_csv(X_train_save_path, index=True)

# Test Data

In [113]:
test_features = make_features_from_df(X_test)

# How many infs
print(f"Number of infs: {np.sum(np.isinf(test_features), axis=0).sum()}")
# Which columns have infs
cols_with_infs = np.where(np.isinf(test_features))[1]
cols_with_infs_unique = np.unique(cols_with_infs)
print(f"Columns with infs: {len(cols_with_infs_unique)}")
print(
    f"biggest pos value except inf: {np.max(test_features[test_features != np.inf].max())}"
)
print(f"biggest neg value: {np.min(test_features).min()}")
biggest_pos = np.max(test_features[test_features != np.inf].max())
biggest_neg = np.min(test_features).min()
# Replace infs with biggest pos value
test_features[test_features == np.inf] = biggest_pos
# Replace -infs with biggest neg value
test_features[test_features == -np.inf] = biggest_neg

  0%|          | 0/3411 [00:00<?, ?it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9658 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 11998 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
  0%|          | 2/3411 [00:00<02:57, 19.22it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9592 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9420 missing data points in your signal. Filling missing values by using

Problem with feature extraction: wrong number of features
0
[]
[]


/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 14835 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9753 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9425 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/neurokit2/ecg/ecg_clean.py:101: NeuroKitWarning: There are 9748 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
/Users/ericschreiber/miniconda3/envs/ml/lib/python3

Number of infs: 1
Columns with infs: 1
biggest pos value except inf: 400055.98318205395
biggest neg value: -119.9979975954895


/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)


In [114]:
X_test_save_path = os.path.join(
    DATAPATH, "feature_extraction/spectral_analysis_X_test_features.csv"
)
test_features.to_csv(X_test_save_path, index=True)